In [1]:
#import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn import metrics

In [2]:
#Reading dataset
df=pd.read_csv('MF.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Columns: 150 entries, Cate_name to Yield
dtypes: int64(148), object(2)
memory usage: 79.8+ KB


In [3]:
#Building descriptors
X = df.drop(columns=['Yield', 'Cate_name', 'P_name' ])
print('---Descriptors---')
print(X.head())

y = pd.DataFrame(df['Yield'],columns=['Yield'])
print('---Objective---')
print(y.head())

---Descriptors---
   Cate_MF_9  Cate_MF_33  Cate_MF_58  Cate_MF_68  Cate_MF_80  Cate_MF_102  \
0          0           0           0           0           0            0   
1          0           0           0           0           0            0   
2          0           0           0           0           0            0   
3          0           0           0           0           0            0   
4          0           0           0           0           0            0   

   Cate_MF_105  Cate_MF_114  Cate_MF_125  Cate_MF_142  ...  P_MF_1154  \
0            0            0            0            0  ...          1   
1            0            0            0            0  ...          0   
2            0            0            0            0  ...          0   
3            0            0            0            0  ...          0   
4            0            0            0            1  ...          1   

   P_MF_1199  P_MF_1272  P_MF_1380  P_MF_1570  P_MF_1689  P_MF_1750  \
0        

In [4]:
#storage 
data_r2_train =[]
data_RMSE_train = []
data_MAE_train = []
data_r2_test = []
data_RMSE_test = []
data_MAE_test = []
best_model_parametors = []
data_cv_score = []

#SVM_a_X,log(y)
for i in range(10):
    seed=i
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)
    
    #autoscaling
    a_X_train = (X_train - X_train.mean(axis=0)) / X_train.std(axis=0, ddof=1)
    a_X_test = (X_test - X_train.mean(axis=0)) / X_train.std(axis=0, ddof=1)
    a_X_train = a_X_train.dropna(how='any', axis=1)
    a_X_test = a_X_test[a_X_train.columns]
    
    log_y_train = np.log(y_train)
    
    param_svm={'C':[0.1,1,5,10,50,100,200,300,400,500,750,1000],
               'gamma':[100,10,1,0.1,0.01,0.001,0.0001,0.00001]}
    reg_svm = GridSearchCV(SVR(kernel='rbf', epsilon=0.3), param_grid=param_svm, cv=5, n_jobs=16)
    reg_svm.fit(a_X_train,log_y_train['Yield'])
    reg_best = reg_svm.best_estimator_
    y_pred1 = np.exp(reg_best.predict(a_X_train))
    y_pred2 = np.exp(reg_best.predict(a_X_test))
    
    #train
    r2_train = metrics.r2_score(y_train, y_pred1)
    RMSE_train = metrics.mean_squared_error(y_train, y_pred1)
    MAE_train =  metrics.mean_absolute_error(y_train, y_pred1)
    #test
    r2_test = metrics.r2_score(y_test, y_pred2)
    RMSE_test = metrics.mean_squared_error(y_test, y_pred2)
    MAE_test = metrics.mean_absolute_error(y_test, y_pred2)
    
    parametors = reg_svm.best_params_
    best_model_parametors.append(parametors)
    cv_score = reg_svm.best_score_
    data_cv_score.append(cv_score)
    
    data_r2_train.append(r2_train)
    data_RMSE_train.append(RMSE_train)
    data_MAE_train.append(MAE_train)
    data_r2_test.append(r2_test)
    data_RMSE_test.append(RMSE_test)
    data_MAE_test.append(MAE_test)
    
    print('----------------------')
    print('seed:', seed)
    print("Best Model Parameter:",reg_svm.best_params_)
    print("Best Model Score:",reg_svm.best_score_)
    print('R2_test:', r2_test)
print('R2_train_means:', sum(data_r2_train)/10)
print('CV_score_means:', sum(data_cv_score)/10)
print('R2_test_means:', sum(data_r2_test)/10)

----------------------
seed: 0
Best Model Parameter: {'C': 400, 'gamma': 0.0001}
Best Model Score: 0.9260550342332469
R2_test: 0.7145786410708018
----------------------
seed: 1
Best Model Parameter: {'C': 500, 'gamma': 0.0001}
Best Model Score: 0.9041865864324409
R2_test: 0.8158857731871294
----------------------
seed: 2
Best Model Parameter: {'C': 500, 'gamma': 0.0001}
Best Model Score: 0.9375924700788406
R2_test: 0.7779666434786198
----------------------
seed: 3
Best Model Parameter: {'C': 400, 'gamma': 0.0001}
Best Model Score: 0.9112551077555959
R2_test: 0.811597278159159
----------------------
seed: 4
Best Model Parameter: {'C': 1000, 'gamma': 0.0001}
Best Model Score: 0.9088453330990562
R2_test: 0.8076923069130946
----------------------
seed: 5
Best Model Parameter: {'C': 1000, 'gamma': 1e-05}
Best Model Score: 0.8982972671485936
R2_test: 0.5631014152690298
----------------------
seed: 6
Best Model Parameter: {'C': 1000, 'gamma': 0.0001}
Best Model Score: 0.8915067825705496
R2_te

In [5]:
data_r2_train_pd = pd.DataFrame(data= data_r2_train, columns=['r2_train'])
data_RMSE_train_pd = pd.DataFrame(data=data_RMSE_train, columns=['RMSE_train'])
data_MAE_train_pd = pd.DataFrame(data=data_MAE_train, columns=['MAE_train'])
data_r2_test_pd = pd.DataFrame(data=data_r2_test, columns=['r2_test'])
data_RMSE_test_pd = pd.DataFrame(data=data_RMSE_test, columns=['RMSE_test'])
data_MAE_test_pd = pd.DataFrame(data=data_MAE_test, columns=['MAE_test'])
data_cv_score_pd = pd.DataFrame(data=data_cv_score, columns=['cv_score'])
data_parametors = pd.DataFrame([best_model_parametors])

data_all = pd.concat([data_r2_train_pd, data_RMSE_train_pd, data_MAE_train_pd, data_r2_test_pd, 
                      data_RMSE_test_pd, data_MAE_test_pd, data_cv_score_pd],
                     axis=1, join='inner')

data_all.loc['mean'] = data_all.mean()
print(data_all)

data_all.to_csv('../../score/MF//SVM_score.csv')

      r2_train  RMSE_train  MAE_train   r2_test   RMSE_test   MAE_test  \
0     0.910632   65.887895   5.078960  0.714579  373.316899  11.949155   
1     0.875201  112.856665   6.880168  0.815886  178.378714   7.822454   
2     0.893646   98.387918   6.366829  0.777967  214.719850   8.833292   
3     0.881712  110.328667   6.779024  0.811597  180.818765   7.983020   
4     0.900697   85.608671   5.922603  0.807692  204.485436   8.787850   
5     0.789110  170.765594   7.000468  0.563101  531.357842  14.212411   
6     0.884157   95.069298   6.682697  0.630042  445.230128  13.479304   
7     0.898184   97.386943   6.562042  0.759071  218.403663   9.168385   
8     0.894883   92.515184   5.993947  0.653513  342.410493  12.630056   
9     0.850543  151.466493   8.327877  0.871654   90.440107   5.849256   
mean  0.877877  108.027333   6.559462  0.740510  277.956190  10.071518   

      cv_score  
0     0.926055  
1     0.904187  
2     0.937592  
3     0.911255  
4     0.908845  
5     0.8